In [4]:
import pyspark
from pyspark.context import SparkContext
from pyspark import SparkConf
from numpy import array
from pyspark.mllib.clustering import KMeans, KMeansModel
# conf = SparkConf()
# sc = SparkContext(conf = conf)
# sc.setLogLevel("ERROR")
f = sc.textFile("ionosphere_libsvm.txt")
firstline = f.first()
data = f.filter(lambda x: x!= firstline).map(lambda line:line.split(",")).map(lambda x:[float(y) if type(y) is float else y for y in x])

model = KMeans.train(data.map(lambda x:x[1:]), 3, maxIterations=100, initializationMode="random")
clustered = data.map(lambda x:(model.predict(array(x[1:])),x))
clusters = clustered.map(lambda p_d:(p_d[0],hash(tuple(p_d[1][1:])))).groupByKey().map(lambda x_y:(x_y[0],set(x_y[1])))
print(clusters.first())
ground = data.map(lambda x:(x[0],hash(tuple(x[1:])))).groupByKey().map(lambda x_y1:(x_y1[0],set(x_y1[1])))
print(ground.first())
#initialization 
obtained = []
for k,v in clusters.collect():
    obtained.append(v)
truth = []
for k,v in ground.collect():
    truth.append(v)

#This is my definition of measure, with its change, the results may change
def measure(s,t):
    #s,t are sets
    left = float(len(s-t))/len(s)
    right = float(len(t-s))/len(t)
    return left+right 
total = data.count()
misplaced = 0

while truth and obtained: 
    minimum = total
    for i,s in enumerate(truth):
        for j,t in enumerate(obtained):
            m = measure(s,t)
            if minimum > m:
                minimum = m
                assignment = (i,j)
    misplaced += len(obtained[assignment[1]]-truth[assignment[0]])

    del obtained[assignment[1]]
    del truth[assignment[0]]
print("% of correctly placed points = ",(float(total-misplaced)/total)*100)#,misplaced
sc.stop()

(0, {3527539})
('1 1:1 3:1 4:-0.02401 5:0.9414 6:0.06531 7:0.92106 8:-0.23255 9:0.77152 10:-0.16399 11:0.52798 12:-0.20275 13:0.56409 14:-0.00712 15:0.34395 16:-0.27457 17:0.5294 18:-0.2178 19:0.45107 20:-0.17813 21:0.05982 22:-0.35575 23:0.02309 24:-0.52879 25:0.03286 26:-0.65158 27:0.1329 28:-0.53206 29:0.02431 30:-0.62197 31:-0.05707 32:-0.59573 33:-0.04608 34:-0.65697 ', {3527539})
% of correctly placed points =  100.0
